## 환경설정

In [1]:
from dotenv import load_dotenv

# .env 파일을 만들고, OpenAI api key 를 붙여넣기합니다.
# OPENAI_API_KEY=sk-

# 토큰 정보로드
load_dotenv()

True

## PDF 기반 질의 응답(Question-Answering)Permalink

### 데이터 로드

In [2]:
from langchain.document_loaders import PyPDFLoader

# 약관문서 샘플 : https://www.kbinsure.co.kr/CG802030002.ec
# 20240401 일자 버전.
# PDF 파일 로드
loader = PyPDFLoader("data/[일반보험]_KB개인상해보험_보험약관.pdf")
document = loader.load()
document[0].page_content[:200] # 내용 추출

'KB개인상해보험'

### 데이터 분할

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

### 저장 및 검색

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 임베딩
embeddings = OpenAIEmbeddings()

# 저장할 경로 지정
DB_PATH = "./chroma_db"

# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings, persist_directory=DB_PATH, collection_name="my_db")
# retriever 가져옴
retriever = docsearch.as_retriever()

C:\Users\RainbowBrain\AppData\Local\Temp\ipykernel_9828\1140094544.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


### 프롬프트 템플릿

In [5]:
# langchain hub 에서 Prompt 다운로드 예시
# https://smith.langchain.com/hub/rlm/rag-prompt

from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

c:\Users\RainbowBrain\rag_note\.venv\lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### 생성

In [6]:
# LLM
from langchain.chat_models import ChatOpenAI

# ChatGPT 모델 지정
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)

C:\Users\RainbowBrain\AppData\Local\Temp\ipykernel_9828\1908171649.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)


In [7]:
# RAG chain 생성
from langchain.schema.runnable import RunnablePassthrough

# pipe operator를 활용한 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

### 테스트

In [8]:
rag_chain.invoke("이 문서의 제목은 뭐야?")

AIMessage(content='이 문서의 제목은 "[일반보험]_KB개인상해보험_보험약관.pdf"입니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 3267, 'total_tokens': 3306, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a0eaa5e-39c5-473b-96f9-2dfffaee0d68-0')

In [9]:
rag_chain.invoke("보험금의 지급사유를 알려 줘")

AIMessage(content='보험금의 지급사유는 보험 약관에 따라 다르지만, 일반적으로 피보험자가 사고를 당하거나 손해를 입었을 때 지급됩니다. 그러나 피보험자나 계약자의 고의, 피보험자가 사고 후 도주, 자동차를 경기용이나 경기를 위한 연습용 또는 시험용으로 운전하던 중 사고, 도로교통법에 정한 음주․무면허 상태 또는 약물 상태에서 운전하던 중 사고, 자가용의 경우 피보험자가 자동차를 영업목적으로 운전하던 중 발생한 사고 등의 경우에는 보험금을 지급하지 않습니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 248, 'prompt_tokens': 4906, 'total_tokens': 5154, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d1282e2-2235-4d84-9c39-f89f012860ad-0')

In [10]:
docsearch.get()

{'ids': ['e5ea16d9-1075-4008-92c4-f66a529c7a3f',
  '6a76de34-a21b-46e4-b3e0-f104feb84fb8',
  '2a43e5e6-6c97-4e98-a0d7-16087825a3c3',
  '172a5815-317c-48c8-b3be-cde410e87c24',
  'da0bb55d-15cc-4875-bd4d-76b494a9444b',
  '73c7e851-c70a-4f3f-ae2c-fa03ddec0b10',
  '511525c1-2225-4c21-a7c0-4a8a603e2246',
  '552e67b2-9dad-4706-863e-84eea4700372',
  '1358ed1b-b47c-4e84-a7fe-9746d09492c5',
  'd8bce70b-32ec-470e-880a-89fb633754b6',
  'dec1c5e9-8f07-43ca-9966-56ee018b2552',
  '458c6f0a-b68b-4648-befd-aa7c0b458505',
  '43076992-3b64-4420-82ad-4e2ef143688b',
  '3188b057-e243-4c36-ab2e-3dbbd974be06',
  'd511df47-f498-41dc-a7b5-4a87b9f9b554',
  'ee808b20-a0a6-4da8-bc63-2e74c243fbf5',
  'a5c5a3af-6df9-4e6e-9c4f-8c8491c60085',
  '18d7bc96-34cd-4322-af8b-d2ee81803aa7',
  'aeca39f5-3d46-4096-9277-6652ae5e772f',
  '38e61ac3-b543-41e9-892d-4b280e85dbf2',
  '769bc1e7-27d9-4d43-87fd-a30ccb3ea078',
  'fa9f86a4-c73f-4f59-a0c4-2ed76d73c450',
  '0285be43-91a6-4902-8ff1-7a335cf13329',
  '652d7f40-06b2-4563-80e2-